In [ ]:
%%writefile mandelbrot.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <math.h>

#define WIDTH 1920
#define HEIGHT 1080
#define MAX_ITER 256

// Color palette for beautiful visualization
struct Color {
    unsigned char r, g, b;
};

// Generate smooth color gradient
__host__ __device__ Color getColor(int iter, int max_iter) {
    if (iter == max_iter) {
        return {0, 0, 0};  // Black for points in the set
    }

    // Smooth coloring using continuous iteration count
    float t = (float)iter / max_iter;

    Color c;
    // Create a rainbow gradient
    c.r = (unsigned char)(9 * (1 - t) * t * t * t * 255);
    c.g = (unsigned char)(15 * (1 - t) * (1 - t) * t * t * 255);
    c.b = (unsigned char)(8.5 * (1 - t) * (1 - t) * (1 - t) * t * 255);

    return c;
}

// CPU implementation - sequential computation
void mandelbrotCPU(Color *image, int width, int height,
                   double centerX, double centerY, double zoom, int maxIter) {
    double scale = 4.0 / (zoom * width);

    for (int py = 0; py < height; py++) {
        for (int px = 0; px < width; px++) {
            // Map pixel to complex plane
            double x0 = centerX + (px - width / 2.0) * scale;
            double y0 = centerY + (py - height / 2.0) * scale;

            double x = 0.0;
            double y = 0.0;
            int iter = 0;

            // Iterate: z = z^2 + c
            while (x*x + y*y <= 4.0 && iter < maxIter) {
                double xtemp = x*x - y*y + x0;
                y = 2*x*y + y0;
                x = xtemp;
                iter++;
            }

            image[py * width + px] = getColor(iter, maxIter);
        }
    }
}

// GPU kernel - parallel computation
__global__ void mandelbrotGPU(Color *image, int width, int height,
                               double centerX, double centerY, double zoom, int maxIter) {
    int px = blockIdx.x * blockDim.x + threadIdx.x;
    int py = blockIdx.y * blockDim.y + threadIdx.y;

    if (px >= width || py >= height) return;

    double scale = 4.0 / (zoom * width);

    // Map pixel to complex plane
    double x0 = centerX + (px - width / 2.0) * scale;
    double y0 = centerY + (py - height / 2.0) * scale;

    double x = 0.0;
    double y = 0.0;
    int iter = 0;

    // Iterate: z = z^2 + c
    while (x*x + y*y <= 4.0 && iter < maxIter) {
        double xtemp = x*x - y*y + x0;
        y = 2*x*y + y0;
        x = xtemp;
        iter++;
    }

    image[py * width + px] = getColor(iter, maxIter);
}

// Save image as PPM file (can be opened with most image viewers)
void savePPM(const char *filename, Color *image, int width, int height) {
    FILE *fp = fopen(filename, "wb");
    fprintf(fp, "P6\n%d %d\n255\n", width, height);
    fwrite(image, sizeof(Color), width * height, fp);
    fclose(fp);
    printf("Saved: %s\n", filename);
}

int main() {
    printf("===========================================\n");
    printf("MANDELBROT SET: CPU vs GPU Performance\n");
    printf("Resolution: %dx%d (%d megapixels)\n", WIDTH, HEIGHT, (WIDTH*HEIGHT)/1000000);
    printf("===========================================\n\n");

    // Set output directory (works for both local and Google Drive)
    // For Google Colab with mounted Drive, use: /content/drive/MyDrive/mandelbrot/
    // For local execution, use: ./
    const char *outputDir = "./";

    // Check if running in Google Colab with mounted Drive
    FILE *testDrive = fopen("/content/drive/MyDrive/test", "w");
    if (testDrive != NULL) {
        fclose(testDrive);
        remove("/content/drive/MyDrive/test");
        outputDir = "/content/drive/MyDrive/mandelbrot/";
        printf("Google Drive detected! Saving to: %s\n", outputDir);

        // Create output directory (ignore error if exists)
        system("mkdir -p /content/drive/MyDrive/mandelbrot");
    } else {
        printf("Saving to local directory: %s\n", outputDir);
    }
    printf("\n");

    size_t imageSize = WIDTH * HEIGHT * sizeof(Color);

    // Allocate host memory
    Color *h_image_cpu = (Color*)malloc(imageSize);
    Color *h_image_gpu = (Color*)malloc(imageSize);

    // Zoom sequence parameters
    double centerX = -0.7;      // Interesting location in the set
    double centerY = 0.0;
    double zoomStart = 1.0;
    double zoomEnd = 1000.0;
    int numFrames = 5;

    printf("Rendering zoom sequence from %fx to %fx zoom...\n\n", zoomStart, zoomEnd);

    // ============================================
    // CPU RENDERING
    // ============================================
    printf("CPU Rendering:\n");
    printf("------------------------------------------\n");

    clock_t totalCPUTime = 0;

    for (int frame = 0; frame < numFrames; frame++) {
        double zoom = zoomStart * pow(zoomEnd / zoomStart, (double)frame / (numFrames - 1));

        clock_t start = clock();
        mandelbrotCPU(h_image_cpu, WIDTH, HEIGHT, centerX, centerY, zoom, MAX_ITER);
        clock_t end = clock();

        double frameTime = ((double)(end - start)) / CLOCKS_PER_SEC;
        totalCPUTime += (end - start);

        printf("Frame %d (zoom: %.1fx): %.3f seconds\n", frame + 1, zoom, frameTime);

        // Save first and last frame
        if (frame == 0 || frame == numFrames - 1) {
            char filename[200];
            sprintf(filename, "%smandelbrot_cpu_frame%d.ppm", outputDir, frame + 1);
            savePPM(filename, h_image_cpu, WIDTH, HEIGHT);
        }
    }

    double avgCPUTime = ((double)totalCPUTime / CLOCKS_PER_SEC) / numFrames;
    printf("\nAverage CPU time per frame: %.3f seconds\n", avgCPUTime);
    printf("CPU FPS: %.2f\n\n", 1.0 / avgCPUTime);

    // ============================================
    // GPU RENDERING
    // ============================================
    printf("GPU Rendering:\n");
    printf("------------------------------------------\n");

    // Allocate device memory
    Color *d_image;
    cudaMalloc(&d_image, imageSize);

    // Setup kernel launch parameters
    dim3 threadsPerBlock(16, 16);
    dim3 blocksPerGrid((WIDTH + 15) / 16, (HEIGHT + 15) / 16);

    printf("Grid: %dx%d blocks, Block: %dx%d threads\n",
           blocksPerGrid.x, blocksPerGrid.y, threadsPerBlock.x, threadsPerBlock.y);
    printf("Total GPU threads: %d\n\n",
           blocksPerGrid.x * blocksPerGrid.y * threadsPerBlock.x * threadsPerBlock.y);

    // Create CUDA events for timing
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);

    float totalGPUTime = 0.0f;

    for (int frame = 0; frame < numFrames; frame++) {
        double zoom = zoomStart * pow(zoomEnd / zoomStart, (double)frame / (numFrames - 1));

        cudaEventRecord(start);

        mandelbrotGPU<<<blocksPerGrid, threadsPerBlock>>>(
            d_image, WIDTH, HEIGHT, centerX, centerY, zoom, MAX_ITER);

        cudaEventRecord(stop);
        cudaEventSynchronize(stop);

        float frameTime;
        cudaEventElapsedTime(&frameTime, start, stop);
        totalGPUTime += frameTime;

        printf("Frame %d (zoom: %.1fx): %.3f seconds\n", frame + 1, zoom, frameTime / 1000.0);

        // Copy result back and save first and last frame
        if (frame == 0 || frame == numFrames - 1) {
            cudaMemcpy(h_image_gpu, d_image, imageSize, cudaMemcpyDeviceToHost);
            char filename[200];
            sprintf(filename, "%smandelbrot_gpu_frame%d.ppm", outputDir, frame + 1);
            savePPM(filename, h_image_gpu, WIDTH, HEIGHT);
        }
    }

    float avgGPUTime = (totalGPUTime / 1000.0) / numFrames;
    printf("\nAverage GPU time per frame: %.3f seconds\n", avgGPUTime);
    printf("GPU FPS: %.2f\n\n", 1.0 / avgGPUTime);

    // ============================================
    // PERFORMANCE COMPARISON
    // ============================================
    printf("===========================================\n");
    printf("PERFORMANCE SUMMARY\n");
    printf("===========================================\n");
    printf("Average CPU time: %.3f seconds (%.2f FPS)\n", avgCPUTime, 1.0 / avgCPUTime);
    printf("Average GPU time: %.3f seconds (%.2f FPS)\n", avgGPUTime, 1.0 / avgGPUTime);
    printf("Speedup:          %.2fx faster on GPU\n", avgCPUTime / avgGPUTime);
    printf("===========================================\n\n");

    if (avgGPUTime < 0.033) {
        printf("✓ GPU can render at 30+ FPS! Real-time zoom is possible!\n");
    } else if (avgGPUTime < 0.016) {
        printf("✓ GPU can render at 60+ FPS! Buttery smooth real-time zoom!\n");
    }

    printf("\nOutput files saved to: %s\n", outputDir);
    printf("- mandelbrot_cpu_frame1.ppm & mandelbrot_cpu_frame%d.ppm (CPU)\n", numFrames);
    printf("- mandelbrot_gpu_frame1.ppm & mandelbrot_gpu_frame%d.ppm (GPU)\n", numFrames);
    printf("\nTo view in Colab:\n");
    printf("from google.colab import files\n");
    printf("from PIL import Image\n");
    printf("img = Image.open('%smandelbrot_gpu_frame1.ppm')\n", outputDir);
    printf("img.show()\n");
    printf("\nOr download to view locally:\n");
    printf("files.download('%smandelbrot_gpu_frame1.ppm')\n", outputDir);

    // Cleanup
    cudaFree(d_image);
    cudaEventDestroy(start);
    cudaEventDestroy(stop);
    free(h_image_cpu);
    free(h_image_gpu);

    return 0;
}

/*
 * To compile and run:
 *
 * LOCAL:
 * nvcc -o mandelbrot mandelbrot.cu
 * ./mandelbrot
 *
 * GOOGLE COLAB:
 * First mount Google Drive:
 * from google.colab import drive
 * drive.mount('/content/drive')
 *
 * Then compile and run:
 * !nvcc -arch=sm_75 -o mandelbrot mandelbrot.cu
 * !./mandelbrot
 *
 * Files will automatically save to /content/drive/MyDrive/mandelbrot/
 *
 * To view images in Colab:
 * from PIL import Image
 * img = Image.open('/content/drive/MyDrive/mandelbrot/mandelbrot_gpu_frame1.ppm')
 * display(img)
 *
 * Expected performance:
 * - CPU: Several seconds per frame (0.5-3 FPS)
 * - GPU: Milliseconds per frame (30-300+ FPS)
 * - Speedup: 50-500x depending on hardware
 *
 * Why this demonstrates GPU power:
 * - 2+ million pixels computed independently
 * - Same iterative calculation for each pixel
 * - Perfect data parallelism
 * - CPU does them one at a time, GPU does them all at once
 * - Visually stunning output shows the computational power
 *
 * For presentation:
 * You can generate a full zoom sequence by increasing numFrames
 * and create a video from the frames to show smooth real-time
 * zooming that would be impossible on CPU alone.
 */

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!nvcc -arch=sm_75 -o mandelbrot mandelbrot.cu
!./mandelbrot